# Electric Vechicle Charging Stations in California

### MUSA 550: Geospatial Data Science in Python | Myron Bañez, Mia Cherayil, Kendra Hills

### In further refining our analysis, we will look at EV charging stations in the state of California. The data is taken from the U.S. Department of Energy, where we further filtered down the dataset to California, and fuel type electric.

#### Data sources: 
- https://afdc.energy.gov/fuels/electricity_locations.html#/analyze?fuel=ELEC
- https://data.ca.gov/dataset/ca-geographic-boundaries
- https://geohub.lacity.org/datasets/lacounty::hud-qualified-census-tracts-2022/explore

## Importing station and city data

### Our EV data and city shapefiles are imported into notebook, where we further clean our EV station data.

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Point
import geoviews as gv
import geoviews.tile_sources as gvts
from matplotlib import pyplot as plt
import seaborn as sns

import time 
import hvplot.pandas
import holoviews as hv
import esri2gpd
import carto2gpd
import cenpy
pd.options.display.max_columns = 999

colors2 = ['#5ebaff', '#00faf4', '#ffffcc', '#ffe775', '#ffc140', '#ff8f20', '#ff6060']

In [ ]:
stations = pd.read_csv("stations.csv")

stations

In [ ]:
stations_df = stations.drop(['Intersection Directions', 'Plus4', 'Expected Date', 'Cards Accepted', 'BD Blends', 
                               'NG Fill Type Code', 'NG PSI', 'EV Level1 EVSE Num', 'EV Other Info', 'EV Network Web', 'Federal Agency ID', 'Federal Agency Name', 'Hydrogen Status Link',
                               'NG Vehicle Class', 'LPG Primary', 'E85 Blender Pump', 'Intersection Directions (French)',
                               'Access Days Time (French)', 'Groups With Access Code (French)', 'Hydrogen Is Retail',
                               'Access Code', 'Access Detail Code', 'Federal Agency Code', "Facility Type",
                               'CNG Dispenser Num', 'CNG On-Site Renewable Source', 'CNG Total Compression Capacity', 'CNG Storage Capacity', 
                               'LNG On-Site Renewable Source', 'E85 Other Ethanol Blends', 'EV Pricing', 'EV Pricing (French)',
                               'LPG Nozzle Types', 'Hydrogen Pressures', 'Hydrogen Standards', 'CNG Fill Type Code', 'CNG PSI', 
                               'CNG Vehicle Class', 'LNG Vehicle Class', 'EV On-Site Renewable Source', 'Restricted Access', 'BD Blends (French)'], axis=1)
stations_df

In [ ]:
city = gpd.read_file("./places/CA_Places_TIGER2016.shp")
city

city_df = city.rename(columns = {'NAME': 'City'}, inplace=True)

# Data Wrangling

### We then merge the EV station data with the cities and adding a "count" column with a value of 1 in order to add a numeric value to the analysis.

In [ ]:
station_gdf = city.merge(stations_df, on='City')
station_gdf = station_gdf.to_crs(epsg=4326)
station_gdf['Count'] = 1

station_gdf

# Exploratory Analysis

### This exploratory analysis will look at: count of stations by city, count of EV level 2 charging stations, and a further look at stations in Los Angeles.

### The data for the first two analyses is converted from wide to tidy then gathered by the number of stations by city. This data is merged with a prior dataset to retrieve the geometry. To refrain from duplicates, we utilize idxmax then further plot these with hvplot.

### The workflow for stations in Los Angeles is similar to the aforementioned process but includes the additional steps of filtering down to Los Angeles, and merging on street address in order to also see the number of stations by location in the city. This also includes an overlay of HUD qualifying census tracts.

## Count of stations by city

In [ ]:
#Make the Data Tidy
stations_count = pd.melt(
    station_gdf, 
    id_vars=["City","Open Year","geometry"],
    value_vars=["Count"],
    value_name="Value", 
    var_name="Station_Count"
)

stations_count

In [ ]:
#Count by zip code
stations_count_df = stations_count.groupby(['City'])["Value"].sum().reset_index(name='n')
stations_count_df

In [ ]:
stations_count_df_1 = stations_count.merge(stations_count_df, on='City')
stations_count_df_1

In [ ]:
stations_count_df_2 = stations_count_df_1.loc[stations_count_df_1.groupby(['City'])['n'].idxmax()]
stations_count_df_2[250:300]

In [ ]:
choro = stations_count_df_2.hvplot(c='n',
                                 width=1500, 
                                 height=1500, 
                                 alpha=0.5, 
                                 geo=True, 
                                 cmap=colors2, 
                                 hover_cols=['City'])

gvts.CartoDark * choro

### Count of stations by city with EV2 charging stations

In [ ]:
stations_ev2 = pd.melt(
    station_gdf, 
    id_vars=["EV Level2 EVSE Num", "City", "geometry"],
    value_vars=["Count"],
    value_name="Value", 
    var_name="Type"
)

stations_ev2

In [ ]:
stations_ev2_df = stations_ev2.groupby(['City','EV Level2 EVSE Num'])["Value"].sum().reset_index(name='n')
stations_ev2_df

In [ ]:
stations_ev2_df_1 = stations_ev2.merge(stations_ev2_df, on='City')
stations_ev2_df_1

In [ ]:
stations_ev2_df_2 = stations_ev2_df_1.loc[stations_ev2_df_1.groupby(['City'])['n'].idxmax()]
stations_ev2_df_2

In [ ]:
choro2 = stations_ev2_df_2.hvplot(c='n',
                                 width=1500, 
                                 height=1500, 
                                 alpha=0.5, 
                                 geo=True, 
                                 cmap=colors2, 
                                 hover_cols=['City'])

gvts.CartoDark * choro2

### Stations in Los Angeles

In [ ]:
studyarea = [
    "Los Angeles"
]

station_gfd_1 = station_gdf.query("City in @studyarea")

station_gfd_1['geometry'] = gpd.points_from_xy(station_gfd_1['Longitude'], station_gfd_1['Latitude'])
station_gfd_1 = gpd.GeoDataFrame(station_gfd_1, geometry='geometry', crs="EPSG:4326")
station_gfd_1

In [ ]:
stations_la = pd.melt(
    station_gfd_1, 
    id_vars=["Street Address","City", "geometry"],
    value_vars=["Count"],
    value_name="Value", 
    var_name="Type"
)

stations_la[:50]

In [ ]:
stations_la_df = stations_la.groupby(['Street Address'])["Value"].sum().reset_index(name='n')
stations_la_df

In [ ]:
stations_la_df_1 = stations_la.merge(stations_la_df, on='Street Address')
stations_la_df_1

In [ ]:
stations_la_df_2 = stations_la_df_1.loc[stations_la_df_1.groupby(['Street Address'])['n'].idxmax()]
stations_la_df_2

In [ ]:
choro3 = stations_la_df_2.hvplot.points(c='n',
                                 width=1500, 
                                 height=1500, 
                                 alpha=0.5, 
                                 geo=True, 
                                 cmap=colors2, 
                                 hover_cols=['Street Address'])

gvts.CartoDark * choro3

In [ ]:
qct = gpd.read_file("./qct/HUD_Qualified_Census_Tracts_2022.shp")
qct = qct.to_crs(epsg=4326)

qct_yes = [
    "Yes"
]

qct_LA = qct.query("HUD_QCT in @qct_yes")
qct_LA

choro4 = qct_LA.hvplot(c='OBJECTID', 
                             width=1500, 
                             height=1500, 
                             alpha=0.5, 
                             geo=True,
                             color= '#ffffcc', 
                             hover_cols=['TRACTE'])


In [ ]:
gvts.CartoDark * choro4 * choro3

### Stations by Year

In [ ]:
station_year = station_gdf.loc[station_gdf["Open Year"] >2013]
station_year

station_year = pd.melt(
    station_year, 
    id_vars=["Open Year", "City", "geometry"],
    value_vars=["Count"],
    value_name="Value", 
    var_name="Type"
)

station_year

station_year['Open Year'] = station_year['Open Year'].astype(int)

In [ ]:
station_year_df = station_year.groupby(['Open Year'])['Value'].size().reset_index(name='n')
station_year_df

ax = station_year_df.plot.bar(x='Open Year', y='n', rot=0, color='#5ebaff')
plt.xticks(rotation = 45)
plt.title("Count of New Stations By Year in California", fontsize=10)
plt.show()